In [19]:
import MySQLdb
import node_edge_gen as neg
import networkx as nx

from geopy.distance import great_circle
from geopy.distance import vincenty

In [20]:
# DEFINE SQL LOG IN 
HOST = 'localhost'
USER = 'root'
PASSWD = ''
DATABASE = 'bk_map'


In [21]:
#CONNECT TO MySQL
db_connect = MySQLdb.connect(
    host = HOST,
    user = USER,
    passwd = PASSWD,
    db = DATABASE)

cur = db_connect.cursor()

In [168]:
def nearest_node(lat_input, lon_input):
    dis_near=0.05
    find_nearby_nodes = ('''SELECT nodeid, latitude, longitude FROM bk_nodes 
                WHERE latitude BETWEEN %s AND %s
                AND longitude BETWEEN %s AND %s
                ''')
    node_range = (lat_input-dis_near, lat_input+dis_near, lon_input-dis_near, lon_input+dis_near)

    cur.execute(find_nearby_nodes,node_range)  
    rows = cur.fetchall()
    if len(rows) == 0:
        return 'NULL'
    #for row in rows:
    else:
        dist = [rows[0][0],great_circle((lat_input,lon_input),rows[0][1:3]).miles ]
        for row in rows[1:]:
            node_dist =  great_circle((lat_input,lon_input),row[1:3]).miles
            if dist[1] > node_dist:
                dist = [row[0],node_dist]
        return dist[0]

def distance_btw_nodes(node1,node2,node_dic):
    return great_circle(node_dic[node1], node_dic[node2]).miles

def accident_weight_btw_nodes(node1, node2, weight_dic):
##    node1 = str(node1)
##    node2 = str(node2)
    if node1 in weight_dic and node2 in weight_dic:
        return weight_dic[node1] + weight_dic[node2]
    elif node1 in weight_dic and node2 not in weight_dic:
        return weight_dic[node1]
    elif node1 not in weight_dic and node2 in weight_dic:
        return weight_dic[node2]
    else:
        return 0

In [207]:
#SQL graph into nx #MAKE THIS SMALLER BASED ON THE INPUT RANGE
cur.execute('''SELECT nodeid, latitude, longitude FROM bk_nodes
                ''')  
nodes_sql = cur.fetchall()

cur.execute('''SELECT edgeid, node1, node2 FROM bk_edges
                ''')  
edges_sql = cur.fetchall()

cur.execute("""SELECT id, latitude, longitude , n_ped_inj  
    FROM bk_trafficaccidents 
    WHERE n_ped_inj > 0""")
accidents_sql = cur.fetchall()



In [78]:
node_dic = {}
for node in nodes_sql:
    node_dic[node[0]] = node[1:] 

edge_dic = {}
for edge in edges_sql:
    edge_dic[edge[0]]= edge[1:]
    


In [79]:
print accidents_sql[0][1], accidents_sql[0][2]
print accidents_sql[1][1], accidents_sql[1][2]
print accidents_sql[2][1], accidents_sql[2][2]
nearest_node(accidents_sql[2][1],accidents_sql[2][2])

40.673694 -73.9671906
40.6502101 -73.9599766
40.5751488 -73.9855724


'NULL'

In [81]:
acc_dic = {}
for a in range(0,len(accidents_sql)):
    node = nearest_node(accidents_sql[a][1],accidents_sql[a][2])
    acc_dic[accidents_sql[a][0]] = accidents_sql[a][1],accidents_sql[a][2],accidents_sql[a][3], node


In [151]:
#print test_dic42476036 42476712

In [156]:
node1 = 42476036
node2 = 42476712
weight_dic = test_dic
#def accident_weight_btw_nodes(node1, node2, weight_dic):
node1 = str(node1)
node2 = str(node2)
if node1 in weight_dic and node2 in weight_dic:
    print weight_dic[node1] + weight_dic[node2]
elif node1 in weight_dic:
    print weight_dic[node1]
elif node2 in weight_dic:
    print weight_dic[node2]
else:
    print 0

0


In [190]:
def make_weights(node_dict,edge_dict,feature_dict, feature_factor):
    tmp_dic = {}
    for feat in feature_dict:
        node = feature_dict[feat][3]
        if node != 'NULL':
            if node in tmp_dic:
                tmp_dic[node] =  tmp_dic[node] + 1
            else:
                tmp_dic[node] = feature_dict[feat][2]
                
        else:
            pass
    tmp_edge = {}
    for edge in edge_dict:
        #print edge_dict[edge][0], edge_dict[edge][1]
        acc_wei = accident_weight_btw_nodes(edge_dict[edge][0], edge_dict[edge][1], tmp_dic)
        weight = ((feature_factor*acc_wei) + 1)*distance_btw_nodes(edge_dict[edge][0],edge_dict[edge][1],node_dic)
        tmp_edge[edge] = edge_dict[edge][0], edge_dict[edge][1], weight
    return tmp_edge
#returns new edge function with weights 
#feature_factor = 0  ===no accident weightin
#feature_factor === weight you put on the increased distances associated with accidents


test_dic = make_weights(node_dic, edge_dic, acc_dic,5)

In [202]:
def gen_network(node_dic, edge_dic):
    G = nx.Graph()
    for d in node_dic:
        G.add_node(d)
    for e in edge_dic:
        G.add_edge(edge_dic[e][0],edge_dic[e][1],weight = edge_dic[e][2])

    print 'NODES: ', G.number_of_nodes()
    print 'EDGES: ', G.number_of_edges()
    return G

In [203]:
G = gen_network(node_dic, test_dic)

NODES:  2473
EDGES:  3922


In [205]:
import matplotlib.pylab as plt
nx.draw(G, node_dic)
plt.show()

In [208]:
#close database connection
cur.close()
db_connect.close()